<a href="https://colab.research.google.com/github/ChandrashekharRobbi/Speech-Emotion-Detection/blob/main/Speech_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Necessary LIbraries

In [8]:
import pandas as pd
from google.colab import files

In [1]:
!pip install -q kaggle
files.upload()          # expire any previous token(s) and upload recreated token
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download -d ejlok1/toronto-emotional-speech-set-tess

100% 427M/428M [00:20<00:00, 23.6MB/s]
100% 428M/428M [00:20<00:00, 22.4MB/s]


In [3]:
!unzip -q toronto-emotional-speech-set-tess.zip

In [4]:
paths = []
labels = []
import os
for dirname, _, filenames in os.walk('/content/TESS Toronto emotional speech set data'):
  for filename in filenames:
    paths.append(os.path.join(dirname, filename))
    label = filename.split('_')[-1]
    label = label.split('.')[0]
    labels.append(label.lower())
print("Dataset is loaded")

Dataset is loaded


In [5]:
paths[:5]

['/content/TESS Toronto emotional speech set data/OAF_disgust/OAF_wash_disgust.wav',
 '/content/TESS Toronto emotional speech set data/OAF_disgust/OAF_turn_disgust.wav',
 '/content/TESS Toronto emotional speech set data/OAF_disgust/OAF_rot_disgust.wav',
 '/content/TESS Toronto emotional speech set data/OAF_disgust/OAF_said_disgust.wav',
 '/content/TESS Toronto emotional speech set data/OAF_disgust/OAF_fall_disgust.wav']

In [6]:
labels[:5]

['disgust', 'disgust', 'disgust', 'disgust', 'disgust']

In [7]:
len(paths)

2800

## Crete a DataFrame

In [9]:
df = pd.DataFrame()
df["voice"] = paths
df["label"] = labels 

In [10]:
df.head()

,voice,label
0,/content/TESS Toronto emotional speech set dat...,disgust
1,/content/TESS Toronto emotional speech set dat...,disgust
2,/content/TESS Toronto emotional speech set dat...,disgust
3,/content/TESS Toronto emotional speech set dat...,disgust
4,/content/TESS Toronto emotional speech set dat...,disgust


In [11]:
df.tail()

,voice,label
2795,/content/TESS Toronto emotional speech set dat...,ps
2796,/content/TESS Toronto emotional speech set dat...,ps
2797,/content/TESS Toronto emotional speech set dat...,ps
2798,/content/TESS Toronto emotional speech set dat...,ps
2799,/content/TESS Toronto emotional speech set dat...,ps


In [17]:
df["label"].unique()

array(['disgust', 'happy', 'sad', 'angry', 'fear', 'ps', 'neutral'],
      dtype=object)

In [18]:
df["label"].value_counts()

disgust    400
happy      400
sad        400
angry      400
fear       400
ps         400
neutral    400
Name: label, dtype: int64